In [1]:
!pip install dill
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 3.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import numpy as np
import dill
import shap
import tensorflow as tf
import timeit

In [ ]:
import sys
sys.path.append("final_codes/ts-mule_main")
sys.path.append("final_codes/ts-mule/Window_Shap")
import dill

import numpy as np
import pandas as pd
import timeit
import tensorflow as tf
import timeit
import matplotlib.pyplot as plt
import shap
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from tsmule.xai.lime import LimeTS
from tsmule.sampling.segment import WindowSegmentation, MatrixProfileSegmentation, SAXSegmentation
from tsmule.sampling.perturb import Perturbation
from tsmule.xai.evaluation import PerturbationAnalysis
import windowshap

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [5]:
dnn_model = tf.keras.models.load_model('/content/drive/MyDrive/Nadiyah/final_codes/Beijing_Air_Quality_models/beijing_air_2_5_dnn_model.h5')

with open('/content/drive/MyDrive/Nadiyah/final_codes/Beijing_Air_Quality_models/beijing_air_2_5_test_data.dill', 'rb') as f:
    dataset_test = dill.load(f)

In [6]:
sample=dataset_test[0] #taking 10,080 samples
labels=dataset_test[1]

In [ ]:
def predict_fn(x):
    if len(x.shape) == 2:
        prediction = dnn_model.predict(x[np.newaxis, :, :]).ravel()
    else:
        prediction = dnn_model.predict(x).ravel()
    return prediction

### Explanations with SHAP

In [9]:
tic=timeit.default_timer()
shap_exp=shap.DeepExplainer(dnn_model, sample[:3000]) #expected shape (None, 24, 7)

shap_values=shap_exp.shap_values(sample)

print('Total time: ' + str(timeit.default_timer()-tic))

Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


Total time: 1225.26341302


In [10]:
shap_values=np.asarray(shap_values).squeeze()
shap_values.shape

(8347, 24, 7)

In [11]:
with open("Beijing_air_quality_dnn_shap_values.dill", 'wb') as f:
    dill.dump(shap_values, f)

### Explanations with Lime and Window Segmentation

In [ ]:
lasso_classifier = linear_model.Lasso(alpha=0.01)  #faster the model, faster LIME works
per=Perturbation() #perturbation object for LimeTS object, sampler

##### Uniform Segmentation

In [ ]:
#LimeTS object for uniform segmentation
tic = timeit.default_timer()

#segments object, WindowSegmentation object has stationery and exponential segmentations techniques
uniform_seg=WindowSegmentation(partitions=4, win_length=24)
uniform_lime=LimeTS(kernel=lasso_classifier, segmenter=uniform_seg, sampler=per, n_samples=24)
lime_values_uni=[uniform_lime.explain(sample[i], predict_fn, segmentation_method='uniform')
                 for i in range(len(sample))]
print('Total time: ' + str(timeit.default_timer()-tic))

1/1 [==============================] - 0s 18ms/step


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



1/1 [==============================] - 0s 19ms/step


In [ ]:
with open("Beijing_air_quality_dnn_lime_values_uni.dill", 'wb') as f:
    dill.dump(lime_values_uni, f)

##### exponential Window Segmentation

In [ ]:
#LimeTS object for exponential window segmentation
tic = timeit.default_timer()
#segment object, WindowSegmentation has stationery and exponentials segmentation techniques
exp_seg=WindowSegmentation(partitions=4, win_length=24)
exp_lime=LimeTS(kernel=lasso_classifier, segmenter=exp_seg, sampler=per, n_samples=24)
#explainer for LimeTS
lime_values_exp=[exp_lime.explain(sample[i], predict_fn, segmentation_method='exponential')
                 for i in range(len(sample))]

print('Total time: ' + str(timeit.default_timer()-tic))

1/1 [==============================] - 0s 46ms/step


In [ ]:
with open("Beijing_air_quality_dnn_lime_values_exp.dill", 'wb') as f:
    dill.dump(lime_values_exp, f)

### Explanations with LIME and MatrixProfile Segmentation

##### Slopes-Sorted

In [ ]:
#LimeTS object for matrix profile segmentation= slopes-sorted
tic=timeit.default_timer()
#segment object, MatrixProfileSegmentation object has slopes, bins-min, bins-max segmentation techniques
seg_slopes=MatrixProfileSegmentation(partitions=4, win_length=20)

lime_slopes=LimeTS(kernel=lasso_classifier, segmenter=seg_slopes, sampler=per)
lime_values_slopes=[lime_slopes.explain(sample[i], predict_fn,
                                                  segmentation_method='slopes-sorted') for i in
                    range(len(sample))]

print('Total time: ' + str(timeit.default_timer()-tic))

In [ ]:
with open("Beijing_air_quality_dnn_lime_values_slopes.dill", 'wb') as f:
    dill.dump(lime_values_slopes, f)

### Explanations with LIME and SAX Transformations

In [ ]:
#LimeTS object for SAX segmentation
tic=timeit.default_timer()
#create segment object for SAX Transformation
seg_sax=SAXSegmentation(partitions=4, win_length=10)

lime_sax=LimeTS(kernel=lasso_classifier, segmenter=seg_sax, sampler=per, n_samples=24)
lime_values_sax=[lime_sax.explain(sample[i], predict_fn) for i in range(len(sample))]

print('Total time: ' + str(timeit.default_timer()-tic))

1/1 [==============================] - ETA: 0s

In [ ]:
with open("Beijing_air_quality_dnn_lime_values_sax.dill", 'wb') as f:
    dill.dump(lime_values_sax, f)